In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.datasets import load_digits
from matplotlib import font_manager
from matplotlib import gridspec
from math import factorial
import sklearn
import pprint
import re
from sklearn.preprocessing import LabelEncoder
import time
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense
import statsmodels.api as sm


import warnings
warnings.filterwarnings('ignore')

font_fname = 'C:/Windows/Fonts/malgun.ttf'
font_family = font_manager.FontProperties(fname=font_fname).get_name()

plt.rcParams["font.family"] = font_family
plt.rcParams["axes.unicode_minus"] = False


%matplotlib inline

In [ ]:
# pd 데이터프레임 보기 설정
# 행의 표시 수 설정
pd.set_option('display.max_rows', 100)

# 열의 표시 수 설정
pd.set_option('display.max_columns', 100)

### 전처리

In [2]:
house_df = pd.read_csv('apartment_rail.csv')
house_df['공급액(만원)'] = house_df['공급액(만원)'].str.replace(',', '').astype(int)

In [3]:
house_ols_df = house_df.copy()
house_ols_df['입주예정일'] = house_df['입주예정일'].astype(str).str[:4]
house_ols_df['대형건설사'] = house_ols_df['대형건설사'].apply(lambda x: True if x == 'YES' else False)
house_ols_df['법정동주소'] = house_ols_df['법정동주소'].astype(str)

In [16]:
house_ols_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3142 entries, 0 to 3141
Data columns (total 33 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   아파트명         3142 non-null   object 
 1   법정동주소        3142 non-null   object 
 2   위도           3142 non-null   float64
 3   경도           3142 non-null   float64
 4   세대수          3142 non-null   int64  
 5   임대세대수        2908 non-null   float64
 6   최고층          3104 non-null   float64
 7   최저층          3095 non-null   float64
 8   최대공급면적       3142 non-null   float64
 9   최소공급면적       3142 non-null   float64
 10  총아파트동수       3114 non-null   float64
 11  용적률          1827 non-null   float64
 12  건폐율          1764 non-null   float64
 13  세대평균_주차대수    3070 non-null   float64
 14  공급면적         3142 non-null   float64
 15  전용면적         3142 non-null   float64
 16  전용율          3139 non-null   float64
 17  방수           3056 non-null   float64
 18  욕실수          3056 non-null   float64
 19  현관구조  

In [4]:
구_list = ['종로구', '용산구', '성동구', '광진구',
'동대문구',
'중랑구',
'성북구',
'강북구',
'도봉구',
'노원구',
'은평구',
'서대문구',
'마포구',
'양천구',
'강서구',
'구로구',
'금천구',
'영등포구',
'동작구',
'관악구',
'서초구',
'강남구',
'송파구',
'강동구',
'동구',
'미추홀구',
'연수구',
'남동구',
'부평구',
'계양구',
'서구',
'강화군',
'옹진군',
'수원시',
'성남시',
'의정부시',
'안양시',
'부천시',
'광명시',
'평택시',
'동두천시',
'안산시',
'고양시',
'과천시',
'구리시',
'남양주시',
'오산시',
'시흥시',
'군포시',
'의왕시',
'하남시',
'용인시',
'파주시',
'이천시',
'안성시',
'김포시',
'화성시',
'광주시',
'양주시',
'포천시',
'연천군',
'가평군',
'양평군',
'여주시']

영종지구 -> 중구
용인처인구 -> 용인시
오산세교2지구 -> 오산시
양근리 산 -> 양평군
갈현동 일원 과천지식정보타운 -> 과천시
경기도 고양덕양구 덕은동403-2번지 -> 고양시
안양동 190-6번지 -> 안양시
경기도 성남수정구 고등동 -> 성남시

### 광역시도/구

In [5]:
def map_district(address):
    for line in 구_list:
        if line in address:
            return line
    
    if "영종지구" in address:
        return "인천_중구"
    elif "용인처인구" in address:
        return "용인시"
    elif "오산세교2지구" in address:
        return "오산시"
    elif "양근리 산" in address:
        return "양평군"
    elif "갈현동 일원 과천지식정보타운" in address:
        return "과천시"
    elif "경기도 고양덕양구 덕은동403-2번지" in address:
        return "고양시"
    elif "안양동 190-6번지" in address:
        return "안양시"
    elif "경기도 성남수정구 고등동" in address:
        return "성남시"
    elif "장항지구" in address:
        return "고양시"
    elif "중구" in address:
        if "서울" in address:
            return "서울_중구"            
        elif "인천" in address:
            return "인천_중구"
    return ''

In [6]:
house_ols_df['구'] = house_ols_df['법정동주소'].apply(map_district)

In [10]:
house_ols_df['구'].value_counts()

화성시      265
연수구      165
평택시      158
용인시      150
수원시      131
        ... 
연천군        6
서울_중구      6
용산구        5
성동구        4
서대문구       2
Name: 구, Length: 63, dtype: int64

In [19]:
이전_df = pd.read_csv('apartment_20230821.csv', encoding='utf-8')

In [20]:
이전_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3330 entries, 0 to 3329
Data columns (total 43 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   아파트명         3330 non-null   object 
 1   면적           3330 non-null   object 
 2   법정동주소        3328 non-null   object 
 3   도로명주소        1959 non-null   object 
 4   위도           3330 non-null   float64
 5   경도           3330 non-null   float64
 6   세대수          3330 non-null   int64  
 7   임대세대수        3091 non-null   float64
 8   영구임대세대수      3042 non-null   float64
 9   국민임대세대수      3042 non-null   float64
 10  민간임대세대수      3042 non-null   float64
 11  공공임대세대수      3042 non-null   float64
 12  장기임대세대수      3042 non-null   float64
 13  기타임대세대수      3041 non-null   float64
 14  최고층          3285 non-null   float64
 15  최저층          3271 non-null   float64
 16  최대공급면적       3293 non-null   float64
 17  최소공급면적       3293 non-null   float64
 18  총아파트동수       3302 non-null   float64
 19  용적률   

In [114]:
blank_indices = house_ols_df[house_ols_df['구'] == ''].index

# 결과 출력
print(blank_indices)

Int64Index([1797, 1798, 1799, 1800, 2024, 2025, 2026, 2027, 2028, 2039, 2040,
            2799, 2800],
           dtype='int64')


In [115]:
selected_rows_a = house_ols_df.loc[blank_indices]
selected_coordinates = selected_rows_a[['위도', '경도']]

# B_df에서 해당 위도, 경도와 일치하는 첫번째 행을 찾아 법정동주소와 도로명주소를 가져옵니다.
addresses = []
for _, coord in selected_coordinates.iterrows():
    matching_row = 이전_df[(이전_df['위도'] == coord['위도']) & (이전_df['경도'] == coord['경도'])].iloc[0]
    legal_address = matching_row['법정동주소']
    road_address = matching_row['도로명주소']
    addresses.append((legal_address, road_address))

# 결과를 A_df의 '법정동주소' 컬럼에 넣어줍니다.
for i, (legal_address, road_address) in enumerate(addresses):
    house_ols_df.loc[blank_indices[i], '법정동주소'] = road_address
    
house_ols_df['구'] = house_ols_df['법정동주소'].apply(map_district)

### 상권 데이터 추가

In [116]:
상권_df = pd.read_excel('상권_변수.xlsx')

In [117]:
상권_df.head()

,광역시도,시군구,소매,음식,교육,장례식장,보건의료,유원지오락,총인구수
0,서울,종로구,9208,6184,975,11,198,246,141223
1,서울,서울_중구,12053,5521,587,18,253,204,120317
2,서울,용산구,9175,4830,856,9,137,176,740
3,서울,성동구,9516,4310,1034,7,171,276,280707
4,서울,광진구,10989,4955,1016,7,203,556,337258


In [118]:
# 새로운 컬럼들을 house_ols_df에 추가
new_columns = ['광역시도', '소매', '음식', '교육', '장례식장', '보건의료', '유원지오락', '총인구수']
for column in new_columns:
    house_ols_df[column] = None

# 상권_df의 데이터를 house_ols_df에 채워넣음
for idx, row in house_ols_df.iterrows():
    구 = row['구']
    matching_row = 상권_df[상권_df['시군구'] == 구]
    if not matching_row.empty:
        for column in new_columns:
            house_ols_df.at[idx, column] = matching_row.iloc[0][column]

In [144]:
house_ols_df[['소매', '음식', '교육', '장례식장', '보건의료', '유원지오락', '총인구수']]

,소매,음식,교육,장례식장,보건의료,유원지오락,총인구수
0,12030,4934,1344,19,332,431,460141
1,12030,4934,1344,19,332,431,460141
2,12030,4934,1344,19,332,431,460141
3,12030,4934,1344,19,332,431,460141
4,12030,4934,1344,19,332,431,460141
5,12030,4934,1344,19,332,431,460141
6,12030,4934,1344,19,332,431,460141
7,12030,4934,1344,19,332,431,460141
8,12030,4934,1344,19,332,431,460141
9,12030,4934,1344,19,332,431,460141


In [139]:
house_ols_df.to_csv('apartment_rail_v4.csv', index=False)

In [178]:
house_ols_df = pd.read_csv('apartment_20230822.csv')

In [27]:
house_ols_df.keys()

Index(['아파트명', '법정동주소', '위도', '경도', '세대수', '임대세대수', '최고층', '최저층', '최대공급면적',
       '최소공급면적', '총아파트동수', '용적률', '건폐율', '세대평균_주차대수', '공급면적', '전용면적', '전용율',
       '방수', '욕실수', '현관구조', '입주예정연도', '공급액(만원)', '대형건설사', '지하철역', '지하철역_거리',
       '역사명', '노선명_리스트', '1차병원', '2차병원', '3차병원', '공원', '대학', '광역', '기초', '소매',
       '음식', '교육', '장례식장', '보건의료', '유원지오락', '총인구수'],
      dtype='object')

In [179]:
house_ols_df['광역_구'] = house_ols_df['광역'] + house_ols_df['기초']

In [180]:
import ast
# 문자열을 파싱하여 리스트로 변환하는 함수
def parse_list_string(s):
    try:
        return ast.literal_eval(s)
    except (ValueError, SyntaxError):
        return []

# '노선명_문자열' 컬럼의 데이터를 리스트로 변환
house_ols_df['노선명_리스트'] = house_ols_df['노선명_리스트'].apply(parse_list_string)


In [181]:
house_ols_df['노선명_리스트'].value_counts()

[1호선]                     677
[경강선]                     227
[인천지하철 1호선]               215
[경의중앙선]                   203
[수인선]                     188
[인천지하철 2호선]               152
[5호선]                     149
[7호선]                     136
[에버라인]                    129
[분당선]                     121
[4호선]                     106
[의정부선]                     86
[인천국제공항선]                  76
[9호선]                      58
[서해선]                      57
[경춘선]                      55
[진접선]                      54
[2호선]                      45
[6호선]                      43
[김포골드라인]                   39
[5호선, 경의중앙선]               36
[우이신설선]                    30
[신분당선]                     29
[수인선, 4호선]                 24
[8호선]                      23
[우이신설선, 1호선, 2호선]          21
[3호선]                      21
[5호선, 8호선]                 19
[인천지하철 1호선, 1호선]           15
[신림선]                      15
[2호선, 5호선]                 12
[인천국제공항선, 6호선, 경의중앙선]      11
[인천지하철 1호선, 인천국제공항선]        9
[인천지하철 1호선

### 원 핫 인코딩

In [199]:
from sklearn.preprocessing import MultiLabelBinarizer

# 노선명을 추출하여 리스트로 저장
all_routes = [route for routes_list in house_ols_df['노선명_리스트'] for route in routes_list]

# MultiLabelBinarizer를 사용하여 원핫인코딩 수행
mlb = MultiLabelBinarizer()
encoded_routes = mlb.fit_transform(house_ols_df['노선명_리스트'])
encoded_routes_df = pd.DataFrame(encoded_routes, columns=mlb.classes_)

# 원핫인코딩 결과를 원래 데이터프레임과 병합
df_encoded = pd.concat([house_ols_df, encoded_routes_df], axis=1)

df_encoded.keys()

Index(['아파트명', '법정동주소', '위도', '경도', '세대수', '임대세대수', '최고층', '최저층', '최대공급면적',
       '최소공급면적', '총아파트동수', '용적률', '건폐율', '세대평균_주차대수', '공급면적', '전용면적', '전용율',
       '방수', '욕실수', '현관구조', '입주예정연도', '공급액(만원)', '대형건설사', '지하철역', '지하철역_거리',
       '역사명', '노선명_리스트', '1차병원', '2차병원', '3차병원', '공원', '대학', '광역', '기초', '소매',
       '음식', '교육', '장례식장', '보건의료', '유원지오락', '총인구수', '광역_구', '1호선', '2호선',
       '3호선', '4호선', '5호선', '6호선', '7호선', '8호선', '9호선', '경강선', '경의중앙선', '경춘선',
       '김포골드라인', '분당선', '서해선', '수인선', '신림선', '신분당선', '에버라인', '우이신설선', '의정부선',
       '인천국제공항선', '인천지하철 1호선', '인천지하철 2호선', '진접선'],
      dtype='object')

In [183]:
len(df_encoded.keys())

67

In [200]:
df_encoded = pd.get_dummies(df_encoded, columns=['입주예정연도','광역_구'])

In [34]:
len(df_encoded.keys())

136

In [201]:
df_encoded.to_csv('encoded2.csv', index=False)

In [ ]:
df_encoded = pd.read_Csv('encoded2.csv')

In [36]:
print(list(df_encoded.columns))

['아파트명', '법정동주소', '위도', '경도', '세대수', '임대세대수', '최고층', '최저층', '최대공급면적', '최소공급면적', '총아파트동수', '용적률', '건폐율', '세대평균_주차대수', '공급면적', '전용면적', '전용율', '방수', '욕실수', '현관구조', '공급액(만원)', '대형건설사', '지하철역', '지하철역_거리', '역사명', '노선명_리스트', '1차병원', '2차병원', '3차병원', '공원', '대학', '광역', '기초', '소매', '음식', '교육', '장례식장', '보건의료', '유원지오락', '총인구수', '1호선', '2호선', '3호선', '4호선', '5호선', '6호선', '7호선', '8호선', '9호선', '경강선', '경의중앙선', '경춘선', '김포골드라인', '분당선', '서해선', '수인선', '신림선', '신분당선', '에버라인', '우이신설선', '의정부선', '인천국제공항선', '인천지하철 1호선', '인천지하철 2호선', '진접선', '입주예정연도_2020', '입주예정연도_2021', '입주예정연도_2022', '입주예정연도_2023', '입주예정연도_2024', '입주예정연도_2025', '입주예정연도_2026', '입주예정연도_2027', '광역_구_경기가평군', '광역_구_경기고양시', '광역_구_경기과천시', '광역_구_경기광명시', '광역_구_경기광주시', '광역_구_경기구리시', '광역_구_경기군포시', '광역_구_경기김포시', '광역_구_경기남양주시', '광역_구_경기동두천시', '광역_구_경기부천시', '광역_구_경기성남시', '광역_구_경기수원시', '광역_구_경기시흥시', '광역_구_경기안산시', '광역_구_경기안성시', '광역_구_경기안양시', '광역_구_경기양주시', '광역_구_경기양평군', '광역_구_경기여주시', '광역_구_경기연천군', '광역_구_경기오산시', '광역_구_경기용인시', '광역_구_경기의왕시', '광역_구_경기의정부시', '광역_구_경기이

select_list = ['아파트명', '법정동주소', '위도', '경도', '세대수', '임대세대수', '최고층', '최저층', '최대공급면적', '최소공급면적', '총아파트동수', '용적률', '건폐율', '세대평균_주차대수', '공급면적', '전용면적', '전용율', '방수', '욕실수', '현관구조', '공급액(만원)', '대형건설사', '지하철역', '지하철역_거리', '역사명', '노선명_리스트', '1차병원', '2차병원', '3차병원', '공원', '대학', '광역', '기초', '소매', '음식', '교육', '장례식장', '보건의료', '유원지오락', '총인구수', '1호선', '2호선', '3호선', '4호선', '5호선', '6호선', '7호선', '8호선', '9호선', '경강선', '경의중앙선', '경춘선', '김포골드라인', '분당선', '서해선', '수인선', '신림선', '신분당선', '에버라인', '우이신설선', '의정부선', '인천국제공항선', '인천지하철 1호선', '인천지하철 2호선', '진접선', '입주예정연도_2020', '입주예정연도_2021', '입주예정연도_2022', '입주예정연도_2023', '입주예정연도_2024', '입주예정연도_2025', '입주예정연도_2026', '입주예정연도_2027', '광역_구_경기가평군', '광역_구_경기고양시', '광역_구_경기과천시', '광역_구_경기광명시', '광역_구_경기광주시', '광역_구_경기구리시', '광역_구_경기군포시', '광역_구_경기김포시', '광역_구_경기남양주시', '광역_구_경기동두천시', '광역_구_경기부천시', '광역_구_경기성남시', '광역_구_경기수원시', '광역_구_경기시흥시', '광역_구_경기안산시', '광역_구_경기안성시', '광역_구_경기안양시', '광역_구_경기양주시', '광역_구_경기양평군', '광역_구_경기여주시', '광역_구_경기연천군', '광역_구_경기오산시', '광역_구_경기용인시', '광역_구_경기의왕시', '광역_구_경기의정부시', '광역_구_경기이천시', '광역_구_경기파주시', '광역_구_경기평택시', '광역_구_경기포천시', '광역_구_경기하남시', '광역_구_경기화성시', '광역_구_서울강남구', '광역_구_서울강동구', '광역_구_서울강북구', '광역_구_서울강서구', '광역_구_서울관악구', '광역_구_서울광진구', '광역_구_서울구로구', '광역_구_서울노원구', '광역_구_서울도봉구', '광역_구_서울동대문구', '광역_구_서울동작구', '광역_구_서울서대문구', '광역_구_서울서초구', '광역_구_서울성동구', '광역_구_서울성북구', '광역_구_서울송파구', '광역_구_서울양천구', '광역_구_서울영등포구', '광역_구_서울용산구', '광역_구_서울은평구', '광역_구_서울종로구', '광역_구_서울중구', '광역_구_서울중랑구', '광역_구_인천강화군', '광역_구_인천계양구', '광역_구_인천남동구', '광역_구_인천동구', '광역_구_인천미추홀구', '광역_구_인천부평구', '광역_구_인천서구', '광역_구_인천연수구', '광역_구_인천중구']

In [302]:
select_list = ['세대수', '임대세대수', '최고층', '최저층', '최대공급면적', '최소공급면적', '총아파트동수', '용적률', 
               '건폐율', '세대평균_주차대수', '공급면적', '전용면적', '전용율', '방수', '욕실수', '공급액(만원)', '대형건설사',
               '지하철역_거리', '1차병원', '2차병원', '3차병원', '공원', '대학', '소매', '음식', '교육', '장례식장', 
               '보건의료', '유원지오락', '총인구수', '1호선', '2호선', '3호선', '4호선', '5호선', '6호선', '7호선', '8호선', '9호선', '경강선', '경의중앙선', 
               '경춘선', '김포골드라인', '분당선', '서해선', '수인선', '신림선', '신분당선', '에버라인', '우이신설선', '의정부선', '인천국제공항선', 
               '인천지하철 1호선', '인천지하철 2호선', '진접선', '입주예정연도_2020', '입주예정연도_2021', '입주예정연도_2022', '입주예정연도_2023', 
               '입주예정연도_2024', '입주예정연도_2025', '입주예정연도_2026', '입주예정연도_2027', '광역_구_경기가평군', '광역_구_경기고양시', '광역_구_경기과천시', 
               '광역_구_경기광명시', '광역_구_경기광주시', '광역_구_경기구리시', '광역_구_경기군포시', '광역_구_경기김포시', '광역_구_경기남양주시', 
               '광역_구_경기동두천시', '광역_구_경기부천시', '광역_구_경기성남시', '광역_구_경기수원시', '광역_구_경기시흥시', '광역_구_경기안산시', 
               '광역_구_경기안성시', '광역_구_경기안양시', '광역_구_경기양주시', '광역_구_경기양평군', '광역_구_경기여주시', '광역_구_경기연천군', 
               '광역_구_경기오산시', '광역_구_경기용인시', '광역_구_경기의왕시', '광역_구_경기의정부시', '광역_구_경기이천시', '광역_구_경기파주시', 
               '광역_구_경기평택시', '광역_구_경기포천시', '광역_구_경기하남시', '광역_구_경기화성시', '광역_구_서울강남구', '광역_구_서울강동구', 
               '광역_구_서울강북구', '광역_구_서울강서구', '광역_구_서울관악구', '광역_구_서울광진구', '광역_구_서울구로구', '광역_구_서울노원구', 
               '광역_구_서울도봉구', '광역_구_서울동대문구', '광역_구_서울동작구', '광역_구_서울서대문구', '광역_구_서울서초구', '광역_구_서울성동구', 
               '광역_구_서울성북구', '광역_구_서울송파구', '광역_구_서울양천구', '광역_구_서울영등포구', '광역_구_서울용산구', '광역_구_서울은평구', 
               '광역_구_서울종로구', '광역_구_서울중구', '광역_구_서울중랑구', '광역_구_인천강화군', '광역_구_인천계양구', '광역_구_인천남동구', '광역_구_인천동구', 
               '광역_구_인천미추홀구', '광역_구_인천부평구', '광역_구_인천서구', '광역_구_인천연수구', '광역_구_인천중구']

### nan, 타입 처리

In [203]:
df_encoded['임대세대수'] = df_encoded['임대세대수'].fillna(0)

# df의 용적률, 건폐율의 -1값을 nan으로 바꾸기
df_encoded['용적률'] = df_encoded['용적률'].replace(-1, np.nan)
df_encoded['건폐율'] = df_encoded['건폐율'].replace(-1, np.nan)

In [310]:
df_encoded[select_list].isna().sum()

세대수              0
임대세대수            0
최고층             30
최저층             43
최대공급면적           0
최소공급면적           0
총아파트동수          24
용적률            410
건폐율            482
세대평균_주차대수       68
공급면적             0
전용면적             0
전용율              0
방수              82
욕실수             82
공급액(만원)          0
대형건설사            0
지하철역_거리          0
1차병원             0
2차병원             0
3차병원             0
공원               0
대학               0
소매               0
음식               0
교육               0
장례식장             0
보건의료             0
유원지오락            0
총인구수             0
1호선              0
2호선              0
3호선              0
4호선              0
5호선              0
6호선              0
7호선              0
8호선              0
9호선              0
경강선              0
경의중앙선            0
경춘선              0
김포골드라인           0
분당선              0
서해선              0
수인선              0
신림선              0
신분당선             0
에버라인             0
우이신설선            0
의정부선             0
인천국제공항선          0
인천지하철 1호선   

In [299]:
select_df = df_encoded.dropna(subset=['용적률', '건폐율'])

In [311]:
select_df.isna().sum()

세대수            0
임대세대수          0
최고층            0
최저층            0
최대공급면적         0
최소공급면적         0
총아파트동수         0
용적률            0
건폐율            0
세대평균_주차대수      0
공급면적           0
전용면적           0
전용율            0
방수             0
욕실수            0
공급액(만원)        0
대형건설사          0
지하철역_거리        0
1차병원           0
2차병원           0
3차병원           0
공원             0
대학             0
소매             0
음식             0
교육             0
장례식장           0
보건의료           0
유원지오락          0
총인구수           0
1호선            0
2호선            0
3호선            0
4호선            0
5호선            0
6호선            0
7호선            0
8호선            0
9호선            0
경강선            0
경의중앙선          0
경춘선            0
김포골드라인         0
분당선            0
서해선            0
수인선            0
신림선            0
신분당선           0
에버라인           0
우이신설선          0
의정부선           0
인천국제공항선        0
인천지하철 1호선      0
인천지하철 2호선      0
진접선            0
입주예정연도_2020    0
입주예정연도_2021    0
입주예정연도_2022    0
입주예정연도_2023   

In [206]:
select_df = select_df.fillna(-1)

In [303]:
select_df = select_df[select_list]

In [193]:
select_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2624 entries, 0 to 3132
Columns: 126 entries, 세대수 to 광역_구_인천중구
dtypes: bool(1), float64(15), int32(25), int64(8), object(6), uint8(71)
memory usage: 1.0+ MB


In [305]:
select_df['대형건설사'] = select_df['대형건설사'].astype(int)

In [306]:
selected_columns = ['소매', '음식', '교육', '장례식장', '보건의료', '유원지오락', '총인구수']


for column in selected_columns:
    select_df[column] = select_df[column].astype(str).str.replace(',', '').astype(int)

In [307]:
object_columns = select_df.select_dtypes(include=['object']).columns
select_df[object_columns].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2624 entries, 0 to 3132
Empty DataFrame


In [309]:
select_df = select_df.dropna()

In [388]:
select_df.to_csv('encoded_v2_drop.csv', index=False)

### OLS

In [2]:
select_df = pd.read_csv('reg_data_real_final_R2.csv')

['세대수', '최고층', '최저층', '최대공급면적', '최소공급면적', '총아파트동수', '용적률', '건폐율', '세대평균_주차대수', '공급면적',
 '전용면적', '전용율', '방수', '욕실수', '공급액_만원', '대형건설사', '지하철역_거리', '병원1차', '병원2차', '병원3차', '공원',
 '대학', '소매', '음식', '교육', '장례식장', '보건의료', '유원지오락', '총인구수', '투기과열지구_after', '분양가상한제_after',
 '투기과열지구_before', '분양가상한제_before', '시군구내_통근통학', '타시군구_통근통학', '타시군구_시군구내', '평당_공급액',
 '타입', '노선_1호선', '노선_2호선', '노선_3호선', '노선_4호선', '노선_5호선', '노선_6호선', '노선_7호선', '노선_8호선','노선_9호선',
 '경강선', '경의중앙선', '경춘선', '김포골드라인', '분당선', '서해선', '수인선', '신림선', '신분당선', '에버라인', '우이신설선',
 '의정부선', '인천국제공항선', '인천_1호선', '인천_2호선', '진접선', '입주예정연도_2020', '입주예정연도_2021', '입주예정연도_2022',
 '입주예정연도_2023', '입주예정연도_2024', '입주예정연도_2025', '입주예정연도_2026', '입주예정연도_2027', '광역_경기', '광역_서울',
 '광역_인천', '기초_경기_가평군', '기초_경기_고양시', '기초_경기_과천시', '기초_경기_광명시', '기초_경기_광주시', '기초_경기_구리시',
 '기초_경기_군포시', '기초_경기_김포시', '기초_경기_남양주시', '기초_경기_동두천시', '기초_경기_부천시', '기초_경기_성남시',
 '기초_경기_수원시', '기초_경기_시흥시', '기초_경기_안산시', '기초_경기_안성시', '기초_경기_안양시', '기초_경기_양주시',
 '기초_경기_양평군', '기초_경기_여주시', '기초_경기_연천군', '기초_경기_오산시', '기초_경기_용인시', '기초_경기_의왕시',
 '기초_경기_의정부시', '기초_경기_이천시', '기초_경기_파주시', '기초_경기_평택시', '기초_경기_포천시', '기초_경기_하남시',
 '기초_경기_화성시', '기초_서울_강남구', '기초_서울_강동구', '기초_서울_강북구', '기초_서울_강서구', '기초_서울_관악구',
 '기초_서울_광진구', '기초_서울_구로구', '기초_서울_금천구', '기초_서울_노원구', '기초_서울_도봉구', '기초_서울_동대문구',
 '기초_서울_동작구', '기초_서울_서대문구', '기초_서울_서초구', '기초_서울_성동구', '기초_서울_성북구', '기초_서울_송파구',
 '기초_서울_양천구', '기초_서울_영등포구', '기초_서울_용산구', '기초_서울_은평구', '기초_서울_종로구', '기초_서울_중구',
 '기초_서울_중랑구', '기초_인천_강화군', '기초_인천_계양구', '기초_인천_남동구', '기초_인천_동구', '기초_인천_미추홀구',
 '기초_인천_부평구', '기초_인천_서구', '기초_인천_연수구', '기초_인천_중구']

In [3]:
y_col = ['공급액_만원', '평당_공급액']

In [4]:
select_df = select_df.astype(float)

In [12]:
# Extract columns for endog and exog variables
endog = select_df[y_col[0]]
exog_cols = ['세대수', '최고층', '최저층', '최대공급면적', '최소공급면적', '총아파트동수', '용적률', '건폐율', '세대평균_주차대수', '공급면적',
        '전용면적', '전용율', '방수', '욕실수', '대형건설사', '지하철역_거리', '병원1차', '병원2차', '병원3차', '공원',
        '대학', '소매', '음식', '교육', '장례식장', '보건의료', '유원지오락', '총인구수', '투기과열지구_after', '분양가상한제_after',
        '투기과열지구_before', '분양가상한제_before', '시군구내_통근통학', '타시군구_통근통학', '타시군구_시군구내',
        '타입', '노선_1호선', '노선_2호선', '노선_3호선', '노선_4호선', '노선_5호선', '노선_6호선', '노선_7호선', '노선_8호선',
        '노선_9호선', '경강선', '경의중앙선', '경춘선', '김포골드라인', '분당선', '서해선', '수인선', '신림선', '신분당선',
        '에버라인', '우이신설선', '의정부선', '인천국제공항선', '인천_1호선', '인천_2호선', '진접선', '입주예정연도_2020',
        '입주예정연도_2021', '입주예정연도_2022','입주예정연도_2023', '입주예정연도_2024', '입주예정연도_2025', '입주예정연도_2026',
        '입주예정연도_2027', '광역_경기', '광역_서울','광역_인천', '기초_경기_가평군', '기초_경기_고양시', '기초_경기_과천시',
        '기초_경기_광명시', '기초_경기_광주시', '기초_경기_구리시','기초_경기_군포시', '기초_경기_김포시', '기초_경기_남양주시',
        '기초_경기_동두천시', '기초_경기_부천시', '기초_경기_성남시',
        '기초_경기_수원시', '기초_경기_시흥시', '기초_경기_안산시', '기초_경기_안성시', '기초_경기_안양시', '기초_경기_양주시',
        '기초_경기_양평군', '기초_경기_여주시', '기초_경기_연천군', '기초_경기_오산시', '기초_경기_용인시', '기초_경기_의왕시',
        '기초_경기_의정부시', '기초_경기_이천시', '기초_경기_파주시', '기초_경기_평택시', '기초_경기_포천시', '기초_경기_하남시',
        '기초_경기_화성시', '기초_서울_강남구', '기초_서울_강동구', '기초_서울_강북구', '기초_서울_강서구', '기초_서울_관악구',
        '기초_서울_광진구', '기초_서울_구로구', '기초_서울_금천구', '기초_서울_노원구', '기초_서울_도봉구', '기초_서울_동대문구',
        '기초_서울_동작구', '기초_서울_서대문구', '기초_서울_서초구', '기초_서울_성동구', '기초_서울_성북구', '기초_서울_송파구',
        '기초_서울_양천구', '기초_서울_영등포구', '기초_서울_용산구', '기초_서울_은평구', '기초_서울_종로구', '기초_서울_중구',
        '기초_서울_중랑구', '기초_인천_강화군', '기초_인천_계양구', '기초_인천_남동구', '기초_인천_동구', '기초_인천_미추홀구',
        '기초_인천_부평구', '기초_인천_서구', '기초_인천_연수구', '기초_인천_중구']

exog = select_df[exog_cols]

# Add constant column to exog
exog = sm.add_constant(exog)

# Fit the OLS model
model_group = sm.OLS(endog, exog).fit()
print(model_group.summary())

                            OLS Regression Results                            
Dep. Variable:                 공급액_만원   R-squared:                       0.887
Model:                            OLS   Adj. R-squared:                  0.882
Method:                 Least Squares   F-statistic:                     179.6
Date:                Wed, 30 Aug 2023   Prob (F-statistic):               0.00
Time:                        17:21:37   Log-Likelihood:                -30664.
No. Observations:                2853   AIC:                         6.157e+04
Df Residuals:                    2733   BIC:                         6.228e+04
Df Model:                         119                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const         -2.902e+04   7970.785     -3.641

In [13]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
# VIF 계산
vif = pd.DataFrame()
df = select_df[exog_cols]
vif["Variable"] = df.columns
vif["VIF"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
vif

,Variable,VIF
0,세대수,16.646706
1,최고층,5.143391
2,최저층,3.206780
3,최대공급면적,2.865905
4,최소공급면적,3.278157
...,...,...
131,기초_인천_미추홀구,inf
132,기초_인천_부평구,inf
133,기초_인천_서구,inf
134,기초_인천_연수구,inf


In [23]:
# Extract columns for endog and exog variables
endog = select_df[y_col[0]]
exog_cols = ['건폐율']


exog = select_df[exog_cols]

# Add constant column to exog
exog = sm.add_constant(exog)

# Fit the OLS model
model_group = sm.OLS(endog, exog).fit()
print(model_group.summary())

                            OLS Regression Results                            
Dep. Variable:                 공급액_만원   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     18.40
Date:                Thu, 31 Aug 2023   Prob (F-statistic):           1.85e-05
Time:                        15:30:05   Log-Likelihood:                -33760.
No. Observations:                2853   AIC:                         6.752e+04
Df Residuals:                    2851   BIC:                         6.754e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.647e+04   1184.697     47.663      0.0

In [21]:
# Extract columns for endog and exog variables
endog = select_df[y_col[0]]
exog_cols = ['세대수', '최고층', '최저층', '최대공급면적', '최소공급면적', '총아파트동수', '용적률', '건폐율', '세대평균_주차대수', '공급면적',
        '욕실수', '지하철역_거리', '병원1차', '병원3차', '공원',
        '대학', '시군구내_통근통학', '타시군구_통근통학', '타시군구_시군구내',
        '타입', '노선_1호선', '노선_2호선', '노선_3호선', '노선_4호선', '노선_5호선', '노선_6호선', '노선_7호선', '노선_8호선',
        '노선_9호선', '경춘선', '분당선', '서해선', '수인선', '신림선', '신분당선',
        '에버라인', '우이신설선', '의정부선', '인천국제공항선', '인천_2호선', '입주예정연도_2020',
        '입주예정연도_2021', '입주예정연도_2022','입주예정연도_2023', '입주예정연도_2025', '입주예정연도_2026',
        '광역_경기', '광역_서울','광역_인천', '기초_경기_가평군', '기초_경기_과천시',
        '기초_경기_광명시', '기초_경기_구리시','기초_경기_군포시', '기초_경기_김포시', '기초_경기_남양주시',
        '기초_경기_동두천시', '기초_경기_성남시',
        '기초_경기_수원시', '기초_경기_시흥시', '기초_경기_안산시', '기초_경기_안성시', '기초_경기_안양시', '기초_경기_양주시',
        '기초_경기_여주시', '기초_경기_연천군', '기초_경기_오산시', '기초_경기_용인시', '기초_경기_의왕시',
        '기초_경기_의정부시', '기초_경기_이천시', '기초_경기_파주시', '기초_경기_평택시', '기초_경기_포천시', '기초_경기_하남시',
        '기초_서울_강남구', '기초_서울_강동구', '기초_서울_관악구',
        '기초_서울_금천구', '기초_서울_노원구', '기초_서울_도봉구', '기초_서울_동대문구',
        '기초_서울_동작구', '기초_서울_서초구', '기초_서울_성동구', '기초_서울_성북구', '기초_서울_송파구',
        '기초_서울_양천구', '기초_서울_영등포구', '기초_서울_용산구', '기초_서울_은평구', '기초_서울_종로구',
        '기초_서울_중랑구', '기초_인천_계양구', '기초_인천_동구',
        '기초_인천_부평구', '기초_인천_서구', '기초_인천_연수구', '기초_인천_중구']


exog = select_df[exog_cols]

# Add constant column to exog
exog = sm.add_constant(exog)

# Fit the OLS model
model_group = sm.OLS(endog, exog).fit()
print(model_group.summary())

                            OLS Regression Results                            
Dep. Variable:                 공급액_만원   R-squared:                       0.883
Model:                            OLS   Adj. R-squared:                  0.879
Method:                 Least Squares   F-statistic:                     211.7
Date:                Thu, 31 Aug 2023   Prob (F-statistic):               0.00
Time:                        10:54:39   Log-Likelihood:                -30711.
No. Observations:                2853   AIC:                         6.162e+04
Df Residuals:                    2754   BIC:                         6.221e+04
Df Model:                          98                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const        -2.77e+04   3237.723     -8.554      

In [18]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
# VIF 계산
vif = pd.DataFrame()
df = select_df[exog_cols]
vif["Variable"] = df.columns
vif["VIF"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
vif

,Variable,VIF
0,세대수,22.952136
1,최고층,25.349856
2,최저층,13.032158
3,최소공급면적,30.989196
4,총아파트동수,24.025924
5,건폐율,6.105244
6,공급면적,31.102457
7,욕실수,50.534889
8,지하철역_거리,2.195414
9,병원1차,2.547926


### drop전 데이터들로 하기

In [140]:
select_list = ['공급면적', '욕실수', '공급액(만원)', '대형건설사',
               '지하철역_거리', '3차병원', '대학', '1호선', '2호선', '3호선', '4호선', '5호선', '6호선', '7호선', '8호선', '9호선', '경강선', '경의중앙선', 
               '경춘선', '김포골드라인', '분당선', '서해선', '수인선', '신림선', '신분당선', '에버라인', '우이신설선', '의정부선', '인천국제공항선', 
               '인천지하철 1호선', '인천지하철 2호선', '진접선', '입주예정연도_2020', '입주예정연도_2021', '입주예정연도_2022', '입주예정연도_2023', 
               '입주예정연도_2024', '입주예정연도_2025', '입주예정연도_2026', '입주예정연도_2027', '광역_구_경기가평군', '광역_구_경기고양시', '광역_구_경기과천시', 
               '광역_구_경기광명시', '광역_구_경기광주시', '광역_구_경기구리시', '광역_구_경기군포시', '광역_구_경기김포시', '광역_구_경기남양주시', 
               '광역_구_경기동두천시', '광역_구_경기부천시', '광역_구_경기성남시', '광역_구_경기수원시', '광역_구_경기시흥시', '광역_구_경기안산시', 
               '광역_구_경기안성시', '광역_구_경기안양시', '광역_구_경기양주시', '광역_구_경기양평군', '광역_구_경기여주시', '광역_구_경기연천군', 
               '광역_구_경기오산시', '광역_구_경기용인시', '광역_구_경기의왕시', '광역_구_경기의정부시', '광역_구_경기이천시', '광역_구_경기파주시', 
               '광역_구_경기평택시', '광역_구_경기포천시', '광역_구_경기하남시', '광역_구_경기화성시', '광역_구_서울강남구', '광역_구_서울강동구', 
               '광역_구_서울강북구', '광역_구_서울강서구', '광역_구_서울관악구', '광역_구_서울광진구', '광역_구_서울구로구', '광역_구_서울노원구', 
               '광역_구_서울도봉구', '광역_구_서울동대문구', '광역_구_서울동작구', '광역_구_서울서대문구', '광역_구_서울서초구', '광역_구_서울성동구', 
               '광역_구_서울성북구', '광역_구_서울송파구', '광역_구_서울양천구', '광역_구_서울영등포구', '광역_구_서울용산구', '광역_구_서울은평구', 
               '광역_구_서울종로구', '광역_구_서울중구', '광역_구_서울중랑구', '광역_구_인천강화군', '광역_구_인천계양구', '광역_구_인천남동구', '광역_구_인천동구', 
               '광역_구_인천미추홀구', '광역_구_인천부평구', '광역_구_인천서구', '광역_구_인천연수구', '광역_구_인천중구']

In [143]:
df_encoded[select_list].isna().sum()

공급면적            0
욕실수            82
공급액(만원)         0
대형건설사           0
지하철역_거리         0
3차병원            0
대학              0
1호선             0
2호선             0
3호선             0
4호선             0
5호선             0
6호선             0
7호선             0
8호선             0
9호선             0
경강선             0
경의중앙선           0
경춘선             0
김포골드라인          0
분당선             0
서해선             0
수인선             0
신림선             0
신분당선            0
에버라인            0
우이신설선           0
의정부선            0
인천국제공항선         0
인천지하철 1호선       0
인천지하철 2호선       0
진접선             0
입주예정연도_2020     0
입주예정연도_2021     0
입주예정연도_2022     0
입주예정연도_2023     0
입주예정연도_2024     0
입주예정연도_2025     0
입주예정연도_2026     0
입주예정연도_2027     0
광역_구_경기가평군      0
광역_구_경기고양시      0
광역_구_경기과천시      0
광역_구_경기광명시      0
광역_구_경기광주시      0
광역_구_경기구리시      0
광역_구_경기군포시      0
광역_구_경기김포시      0
광역_구_경기남양주시     0
광역_구_경기동두천시     0
광역_구_경기부천시      0
광역_구_경기성남시      0
광역_구_경기수원시      0
광역_구_경기시흥시      0
광역_구_경기안산시      0
광역_구_경기안성시

In [214]:
select_df = df_encoded[select_list].dropna()
select_df['대형건설사'] = select_df['대형건설사'].astype(int)

In [215]:
select_df.isna().sum()

세대수            0
임대세대수          0
최고층            0
최저층            0
최대공급면적         0
최소공급면적         0
총아파트동수         0
용적률            0
건폐율            0
세대평균_주차대수      0
공급면적           0
전용면적           0
전용율            0
방수             0
욕실수            0
공급액(만원)        0
대형건설사          0
지하철역_거리        0
1차병원           0
2차병원           0
3차병원           0
공원             0
대학             0
소매             0
음식             0
교육             0
장례식장           0
보건의료           0
유원지오락          0
총인구수           0
1호선            0
2호선            0
3호선            0
4호선            0
5호선            0
6호선            0
7호선            0
8호선            0
9호선            0
경강선            0
경의중앙선          0
경춘선            0
김포골드라인         0
분당선            0
서해선            0
수인선            0
신림선            0
신분당선           0
에버라인           0
우이신설선          0
의정부선           0
인천국제공항선        0
인천지하철 1호선      0
인천지하철 2호선      0
진접선            0
입주예정연도_2020    0
입주예정연도_2021    0
입주예정연도_2022    0
입주예정연도_2023   

In [216]:
select_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2520 entries, 0 to 3127
Columns: 126 entries, 세대수 to 광역_구_인천중구
dtypes: float64(15), int32(26), int64(8), object(6), uint8(71)
memory usage: 1021.3+ KB


In [277]:
# Extract columns for endog and exog variables
endog = select_df['공급액(만원)']
exog_cols = ['공급면적', '욕실수', '대형건설사','3차병원', '대학','공원',
               '지하철역_거리', '3차병원', '1호선', '2호선', '3호선', '4호선', '5호선', '6호선', '8호선', '9호선', '경강선',
               '분당선', '서해선', '수인선', '신림선', '신분당선', '에버라인', '우이신설선', '인천국제공항선', 
               '인천지하철 2호선', '진접선', '입주예정연도_2020', '입주예정연도_2022', '입주예정연도_2023', 
               '입주예정연도_2025', '입주예정연도_2026', '광역_구_경기가평군', '광역_구_경기과천시', 
               '광역_구_경기광명시', '광역_구_경기광주시', '광역_구_경기김포시', '광역_구_경기남양주시', 
               '광역_구_경기동두천시', '광역_구_경기부천시', '광역_구_경기수원시', '광역_구_경기시흥시', 
               '광역_구_경기안성시', '광역_구_경기안양시', '광역_구_경기양주시', '광역_구_경기양평군', '광역_구_경기여주시', '광역_구_경기연천군', 
               '광역_구_경기오산시', '광역_구_경기용인시', '광역_구_경기의왕시', '광역_구_경기의정부시', '광역_구_경기이천시', '광역_구_경기파주시', 
               '광역_구_경기평택시', '광역_구_경기포천시', '광역_구_경기화성시', '광역_구_서울강남구', '광역_구_서울강동구', 
               '광역_구_서울강북구', '광역_구_서울강서구', '광역_구_서울광진구', '광역_구_서울구로구',
               '광역_구_서울도봉구', '광역_구_서울동대문구', '광역_구_서울동작구', '광역_구_서울서대문구', '광역_구_서울서초구', '광역_구_서울성동구', 
               '광역_구_서울성북구', '광역_구_서울영등포구', 
               '광역_구_서울종로구', '광역_구_서울중구', '광역_구_서울중랑구', '광역_구_인천강화군', '광역_구_인천계양구', '광역_구_인천남동구', '광역_구_인천동구', 
               '광역_구_인천미추홀구', '광역_구_인천부평구', '광역_구_인천서구', '광역_구_인천연수구', '광역_구_인천중구']
exog = select_df[exog_cols]

# Add constant column to exog
exog = sm.add_constant(exog)

# Fit the OLS model
model_group = sm.OLS(endog, exog).fit()
print(model_group.summary())

                            OLS Regression Results                            
Dep. Variable:                공급액(만원)   R-squared:                       0.886
Model:                            OLS   Adj. R-squared:                  0.882
Method:                 Least Squares   F-statistic:                     230.4
Date:                Wed, 23 Aug 2023   Prob (F-statistic):               0.00
Time:                        11:37:42   Log-Likelihood:                -27114.
No. Observations:                2520   AIC:                         5.439e+04
Df Residuals:                    2437   BIC:                         5.488e+04
Df Model:                          82                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -2.245e+04   2215.696    -10.133      

In [278]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
# VIF 계산
vif = pd.DataFrame()
vif["Variable"] = exog.columns
vif["VIF"] = [variance_inflation_factor(exog.values, i) for i in range(exog.shape[1])]
vif

,Variable,VIF
0,const,92.198024
1,공급면적,2.184098
2,욕실수,1.946152
3,대형건설사,1.562117
4,3차병원,inf
5,대학,2.965286
6,공원,1.933620
7,지하철역_거리,2.402143
8,3차병원,inf
9,1호선,5.043139


In [289]:
# Extract columns for endog and exog variables
endog = select_df['공급액(만원)']
exog_cols = ['공급면적', '대형건설사','3차병원', '공원',
               '지하철역_거리', '2호선', '6호선', '9호선', 
               '분당선', '수인선', '신림선', '신분당선', '에버라인', '우이신설선', '인천국제공항선', 
               '인천지하철 2호선', '진접선', '입주예정연도_2020', '입주예정연도_2022', '입주예정연도_2023', 
               '광역_구_경기가평군', '광역_구_경기과천시', 
               '광역_구_경기광명시', '광역_구_경기광주시',
               '광역_구_경기동두천시', '광역_구_경기성남시', '광역_구_경기수원시', 
               '광역_구_경기안성시', '광역_구_경기안양시', '광역_구_경기양주시', '광역_구_경기양평군', '광역_구_경기여주시', '광역_구_경기연천군', 
               '광역_구_경기오산시', '광역_구_경기의왕시', '광역_구_경기의정부시', '광역_구_경기이천시', '광역_구_경기파주시', 
               '광역_구_경기평택시', '광역_구_경기포천시', '광역_구_서울강남구', '광역_구_서울강동구', 
               '광역_구_서울강북구', '광역_구_서울강서구', '광역_구_서울광진구', '광역_구_서울구로구',
               '광역_구_서울도봉구', '광역_구_서울동대문구', '광역_구_서울동작구', '광역_구_서울서대문구', '광역_구_서울서초구', '광역_구_서울성동구', 
               '광역_구_서울영등포구', 
               '광역_구_서울종로구', '광역_구_서울중구', '광역_구_서울중랑구', '광역_구_인천계양구', '광역_구_인천동구', 
               '광역_구_인천연수구']
exog = select_df[exog_cols]

# Add constant column to exog
exog = sm.add_constant(exog)

# Fit the OLS model
model_group = sm.OLS(endog, exog).fit()
print(model_group.summary())

                            OLS Regression Results                            
Dep. Variable:                공급액(만원)   R-squared:                       0.875
Model:                            OLS   Adj. R-squared:                  0.872
Method:                 Least Squares   F-statistic:                     293.1
Date:                Wed, 23 Aug 2023   Prob (F-statistic):               0.00
Time:                        11:41:38   Log-Likelihood:                -27223.
No. Observations:                2520   AIC:                         5.457e+04
Df Residuals:                    2460   BIC:                         5.492e+04
Df Model:                          59                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -2.141e+04   1369.960    -15.625      

In [290]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
# VIF 계산
vif = pd.DataFrame()
vif["Variable"] = exog.columns
vif["VIF"] = [variance_inflation_factor(exog.values, i) for i in range(exog.shape[1])]
vif.sort_values(by='VIF', ascending=False).head()

,Variable,VIF
0,const,32.636763
6,2호선,3.263506
14,우이신설선,2.501729
43,광역_구_서울강북구,2.017622
5,지하철역_거리,1.872004
